In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [26]:
import os

des_arr = np.array([])
for image in os.listdir('train'):
    path = os.path.join('train',image)
    img = cv.imread(path)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    _, des = sift.detectAndCompute(img,None)
    des_arr = des if des_arr.size == 0 else np.vstack((des_arr, des))


In [28]:
#np.savetxt("des_array.csv", des_arr, delimiter=",")

In [31]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=512, random_state=0).fit(des_arr)

In [32]:
from joblib import dump, load
dump(kmeans, 'kmeans.joblib') 


['kmeans.joblib']

In [40]:
def vectorize_image(image:np.array,model,n_clusters=512):
    _,des = sift.detectAndCompute(image,None)
    classes = model.predict(des)
    hist = np.zeros(n_clusters)
    for clss in classes:
        hist[clss] += 1

    hist /= len(classes)
    return hist


In [43]:
db_csv_name = 'db.csv'

In [98]:
import pandas as pd

ind = 0
for image in os.listdir('db'):
    ind += 1
    path = os.path.join('db',image)
    img = cv.imread(path)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    df = pd.DataFrame(data={'uid':ind,'filepath':path,'encoding_vector':[vectorize_image(img,kmeans)]})
    df.set_index('uid', inplace=True)
    df['encoding_vector'] = df['encoding_vector'].apply(lambda x: np.array2string(x, separator=','))
    df.to_csv(db_csv_name, mode='a', header=not os.path.exists(db_csv_name),sep='\t')
    


In [99]:
db = pd.read_csv('db.csv',delimiter='\t')
db['encoding_vector'] = db['encoding_vector'].apply(lambda x: eval(x))


In [100]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5, metric='cosine').fit(db['encoding_vector'])